In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [23]:
df = pd.read_csv("clustering_dataset_01.csv")
df.head()

,-8.69493,-4.08204,0
0,-8.64040,-3.85833,0
1,-7.70667,-1.47470,0
2,-8.81293,-3.27131,0
3,-8.05403,-3.51724,0
4,-8.29488,-3.51365,0


In [28]:
def dist(p1, p2):
    # dist((1,2,3,4), (5,6,7,8)) # should equal 8
    return np.sqrt(sum([(dim[1]-dim[0])**2 for dim in zip(p1, p2)]))

class KMeans:
    
    def __init__(self, n_clusters=8, random_state=None, max_iter=300):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.max_iter = max_iter
    
    def fit(self, data):
        self.cluster_origins = data.sample(self.n_clusters)
        self.labels = [0]*len(data)
        
        
        for _ in range(self.max_iter):
            current_labels = []
            for point in data:
                current_labels.append(self.classify(point))
            
            if current_labels == self.labels:
                break
            
            self.labels = current_labels
            
    
    def classify(self, point):
        label = 0
        least_dist = dist(self.cluster_origins[0], point)
        
        for i, cluster_origin in enumerate(self.cluster_origins):
            distance = dist(cluster_origin, point) 
            if distance < least_dist:
                least_dist = distance
                label = i
                

,-8.69493,-4.08204,0
362,-8.98359,5.17447,4
101,-7.79125,-2.89470,0
980,-9.94369,10.40600,13


In [26]:
# Building block functions:
    
def dist(x1, x2):
    """ Return euclidean distance between x1 and x2 """
    # todo
    pass
def centroid(xList):
    """ Compute centroid of multi-dimensional x data in xList """
    # TODO
    pass
def assignmentDiffers(yCurrent, yPrev):
    """ Return True if yCurrent differs from yPrev """
    # TODO
    pass
# Let's use a class for our K-Means implementation
class KMeans:
    """ Perform k-means clustering """
    
    def __init__(self, k=5):
        self.k = k          # number of clusters
        self.means = None   # means of clusters
        
    def classify(self, x):
        """ Return the index of the cluster closest to the input """
        # TODO
        pass
    
    def train(self, data):
        """ Train model based on data """
        # TODO assign to self.means, one per cluster
        pass
def kmeans(x, k):
    # Use it like this?
    km = KMeans(k = 10)
    km.train(x)
    # can print out km.means to see the fit means
    # can call km.classify([1,2,3,4]) to get cluster index
    
    #The function should return a list the length of x that contains
    # the cluster number (1 - k) for the corresponding x point
    # TODO determine return valu